In [1]:
# import gym
from header import *
from utils import *
from replay_buffer import *
from models import poly_net
from reconstructors import sigpy_solver
from dqn import DQN
from importlib import reload
heg = 192
wid = 144

/opt/anaconda/envs/pyenv/lib/python3.9/site-packages/h5py/__init__.py:36: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


In [2]:
datapath = '/mnt/shared_a/OCMR/OCMR_fully_sampled_images/'
ncfiles = list([])
for file in os.listdir(datapath):
    if file.endswith(".pt"):
        ncfiles.append(file)
loader = ocmrLoader(ncfiles)

In [3]:
class RL_trainer():
    def __init__(self,dataloader,policy,memory,episodes:int=10,eps:float=1e-3,
                 fulldim:int=144,base:int=10,budget:int=50):
        self.dataloader = dataloader
        self.dataloader.reset()
        
        self.policy   = policy
        self.memory   = memory
        self.episodes = episodes
        self.epi = 0
        self.fulldim = fulldim
        self.base = base
        self.budget = budget
        self.eps = eps
        self.training_record = {'loss':[],'grad_norm':[],'q_values_mean':[],'q_values_std':[]}
        self.steps = 1
    def train(self):      
        # run training
        while self.epi < self.episodes:
            print(f'episode {self.epi:.3d} / {self.episodes}')
            mask = mask_naiveRand(self.fulldim,fix=self.base,other=0,roll=False)   # one mask at a time
            while mask.sum() < self.budget + self.base:
                data_source, data_target = self.dataloader.load()
#                 epsilon = _get_epsilon(steps_epsilon, self.options)
                curr_obs = fft_observe(data_source,mask)
                action   = self.policy.get_action(data_source, mask=mask, eps_threshold=self.eps)
                next_obs, reward = self.policy.step(action, data_target, mask)
                self.memory.push(curr_obs, action, next_obs, reward)
                
                ### compare with random policy
                with torch.no_grad():
                    action_rand = self.policy.get_rand_action(mask=mask)
                    _, reward_rand = self.policy.step(action_rand, data_target, mask)
                ########################################################################
                update_results = self.policy.update_parameters()
                if update_results is not None:
                    for key in self.training_record.keys():
                        self.training_record[key].append(update_results[key])
                    curr_loss = update_results['loss']
                    print(f'step: {self.steps:5d}, loss: {curr_loss:.4f}, RL reward: {reward.mean().item():.4f}, Rand reward: {reward_rand.mean().item():.4f}')
                    torch.cuda.empty_cache()
                self.steps += 1
#                 if self.steps % self.options.target_net_update_freq == 0:
#                     self.logger.info("Updating target network.")
#                     self.target_net.load_state_dict(self.policy.state_dict())
            self.dataloader.reset()
            self.epi += 1

In [4]:
memory = ReplayMemory(capacity=10,curr_obs_shape=(3,192,144),next_obs_shape=(1,192,144),batch_size=2,burn_in=2)
model  = poly_net(samp_dim=144)
policy = DQN(model,memory)
trainer = RL_trainer(loader,policy,memory)

current file: fs_0019_3T.pt


In [ ]:
trainer.train()